### Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator #resim işlemek için

In [2]:
tf.__version__

'2.11.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
#overtrain e karşı görüntüleri sabitliyoruz geometrik transformasyon
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
# batches of augmented image data
training_set = train_datagen.flow_from_directory('dataset/training_set',  # this is the target directory
                                                 target_size = (64, 64),# all images will be resized to 64x64
                                                 batch_size = 32,
                                                 class_mode = 'binary') #since we use binary_crossentropy loss, we need binary labels

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3])) 
#32 filtreli 3d kernel relu activation input_Shape = 64X64X3 boyutlandırdığım boyut

### Step 2 - Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# kaç frame olarak alcak 2x2 o yüzden 2 pool_size strides ise shift kaç pixel ile olacağı

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu')) #yüksek tutuyoruz units i

### Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) 
#ikili sınıflandırma yaptığımız için 1 nöron sigmoid ise sınıflandırma için uygun diye

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 39s 149ms/step - loss: 0.6841 - accuracy: 0.5725 - val_loss: 0.6359 - val_accuracy: 0.6495
Epoch 2/25
250/250 [==============================] - 38s 151ms/step - loss: 0.6308 - accuracy: 0.6466 - val_loss: 0.6060 - val_accuracy: 0.6970
Epoch 3/25
250/250 [==============================] - 40s 160ms/step - loss: 0.5962 - accuracy: 0.6811 - val_loss: 0.5509 - val_accuracy: 0.7210
Epoch 4/25
250/250 [==============================] - 39s 156ms/step - loss: 0.5529 - accuracy: 0.7185 - val_loss: 0.5193 - val_accuracy: 0.7480
Epoch 5/25
250/250 [==============================] - 40s 158ms/step - loss: 0.5266 - accuracy: 0.7365 - val_loss: 0.5336 - val_accuracy: 0.7335
Epoch 6/25
250/250 [==============================] - 39s 157ms/step - loss: 0.4984 - accuracy: 0.7605 - val_loss: 0.5697 - val_accuracy: 0.7145
Epoch 7/25
250/250 [==============================] - 39s 157ms/step - loss: 0.4811 - accuracy: 0.7688 - val_loss: 0.5424 - val_ac

## Part 4 - Making a single prediction

load_img: Resmi yükler.

target_size: Eğitimdeki boyutla aynı hale getirir (64x64).

img_to_array: NumPy dizisine çevirir.

expand_dims: Tek resim olduğu için batch boyutunu ekler.

predict: Olasılık tahmini yapar.

class_indices: Modelin hangi etiketi hangi sayıya atadığını gösterir.

In [14]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64)) #resimleri 64 64 yapmıştık o yüzden bunuda yapıyoruz
test_image = image.img_to_array(test_image)  # Resmi NumPy array'e çevir
test_image = np.expand_dims(test_image, axis = 0) # (1, 64, 64, 3) şekline getir
result = cnn.predict(test_image)
training_set.class_indices # {'cats': 0, 'dogs': 1}
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 [==============================] - 0s 157ms/step


In [15]:
print(prediction)

dog


In [16]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64)) #resimleri 64 64 yapmıştık o yüzden bunuda yapıyoruz
test_image = image.img_to_array(test_image)  # Resmi NumPy array'e çevir
test_image = np.expand_dims(test_image, axis = 0) # (1, 64, 64, 3) şekline getir
result = cnn.predict(test_image)
training_set.class_indices # {'cats': 0, 'dogs': 1}
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'
print(prediction)

1/1 [==============================] - 0s 34ms/step
cat


In [19]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_5.jpg', target_size = (64, 64)) #resimleri 64 64 yapmıştık o yüzden bunuda yapıyoruz
test_image = image.img_to_array(test_image)  # Resmi NumPy array'e çevir
test_image = np.expand_dims(test_image, axis = 0) # (1, 64, 64, 3) şekline getir
result = cnn.predict(test_image)
training_set.class_indices # {'cats': 0, 'dogs': 1}
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'
print(prediction)

1/1 [==============================] - 0s 32ms/step
cat
